<a href="https://colab.research.google.com/github/dearthdiggler/docs/blob/main/recreate_this_application_with_a_supabase_db_back_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a significant architectural shift from the standalone Python/Streamlit script described in the text to a modern, decoupled web application.

To achieve the goal of an "always available" and "super fast" feel, we must move away from the current model where data processing happens *when the user loads the page*.

Instead, we will adopt an asynchronous architecture:

1.  **The Backend "Brain" (Python Worker):** A background service running on Fly.io. It runs periodically (e.g., nightly or hourly). It uses Firecrawl, OCR, and other tools to crunch the data, calculate scores, and find leads. It then pushes this "ready-to-serve" data into Supabase.
2.  **The Database (Supabase):** PostgreSQL hosting the cleaned, scored data. It provides an instant API for the frontend.
3.  **The Frontend (Vite/React):** A lightweight web application. When it loads, it doesn't do any math. It just asks Supabase for the pre-calculated JSON data, rendering almost instantly.

Here is the blueprint and essential code blocks for this new architecture.

### The Architecture Diagram

```mermaid
graph TD
    subgraph "Fly.io (Backend)"
        Scheduler[Cron/Scheduler] --> Worker[Python Atlas Worker Container]
        Worker --> Firecrawl[Firecrawl API]
        Worker --> OCR[Tesseract OCR]
        Worker --> Sources[Reddit/Zillow APIs]
        Worker -- "Upserts clean, scored JSON data" --> SupabaseDB
    end

    subgraph "Supabase (BaaS)"
        SupabaseDB[(PostgreSQL DB)]
        Auth[Supabase Auth]
        API[Auto-generated REST/GraphQL API]
    end

    subgraph "Fly.io (Frontend)"
        ReactApp[Vite/React Container] -- "Fetches pre-calculated JSON" --> API
    end

    User[User Browser] --> ReactApp
```

-----

### Phase 1: Supabase Database Schema

[cite\_start]You need to set up your tables in Supabase to match the data structures defined in the Python code[cite: 34, 704]. Run these SQL queries in your Supabase SQL Editor.

```sql
-- Enable PostGIS if you want advanced map features later
create extension if not exists postgis schema extensions;

-- 1. STR Properties Table (The "Registry Data")
[cite_start]-- Based on CANONICAL_COLUMNS [cite: 34]
create table public.str_properties (
    registration_id text primary key,
    address text,
    city text default 'Flagstaff',
    state text default 'AZ',
    zip text,
    latitude float,
    longitude float,
    neighborhood text,
    bedrooms int,
    bathrooms float,
    square_feet int,
    year_built int,
    license_status text,
    occupancy_rate_estimate float,
    adr_estimate float,
    annual_revenue_estimate float,
    review_score float,
    list_price float,
    days_on_market int,
    listing_type text, -- 'MLS', 'FSBO'
    estimated_value float,
    equity_amount float,
    equity_ratio float,
    back_taxes_amount float,
    has_recent_lawsuit boolean,
    -- Atlas Calculated Segments & Scores
    atlas_exploit_score float,
    atlas_segment text, -- 'Distressed', 'Invest', 'Divest', 'Neutral'
    updated_at timestamp with time zone default timezone('utc'::text, now())
);

-- 2. DID Re Leads Table (The "Deal Scout")
[cite_start]-- Based on the SQLAlchemy Lead model [cite: 705]
create table public.leads (
    id bigint generated by default as identity primary key,
    created_at timestamp with time zone default timezone('utc'::text, now()),
    source text not null, -- e.g., 'reddit'
    source_id text not null,
    segment text not null, -- 'Distress', 'Invest', 'Divest'
    agent_name text not null,
    score float,
    title text,
    url text,
    price float,
    contact_handle text,
    status text default 'New', -- New, Contacted, Dead
    last_contacted_at timestamp with time zone,
    times_contacted int default 0,
    unique(source, source_id)
);

-- 3. Compliance Tables
create table public.do_not_contact (
    handle text primary key,
    reason text,
    created_at timestamp with time zone default timezone('utc'::text, now())
);

-- Add Row Level Security (RLS) so the public can't edit your data
alter table public.str_properties enable row level security;
alter table public.leads enable row level security;

-- Policy: Everyone can read (for your frontend)
create policy "Public Read STR" on public.str_properties for select using (true);
create policy "Public Read Leads" on public.leads for select using (true);

-- Policy: Only service_role (your Python worker) can insert/update
-- (You configure this in Supabase Auth settings, not just SQL)
```

-----

### Phase 2: The Python Worker (Backend)

This script runs on Fly.io as a background process. It replaces the Streamlit script. It uses your existing logic but saves to Supabase instead of local files.

**Requirements:** `pandas supabase firecrawl-py pytesseract praw beautifulsoup4 requests`

#### `worker.py` (Conceptual Wrapper)

This orchestrates the ingestion and scouting tools.

In [ ]:
import os
import pandas as pd
from dotenv import load_dotenv
from supabase import create_client, Client

# Import your existing modules (slightly refactored for Supabase)
# from atlas_ingestion import ingest_flagstaff_data
# from deal_scout_didre import run_all_didre_agents

load_dotenv()

# Initialize Supabase Client (requires service_role key for writing)
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_SERVICE_KEY = os.getenv("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(SUPABASE_URL, SUPABASE_SERVICE_KEY)

def run_str_ingestion_job():
    print("Starting STR Ingestion Job...")
    # [cite_start]1. Run the ingestion logic (SQL -> OCR -> Firecrawl) defined in the text [cite: 7-9].
    # [cite_start]2. Run the scoring logic (Distressed/Invest/Divest)[cite: 133].
    # 3. Get the resulting Pandas DataFrame.
    # df = ingest_flagstaff_data()

    # MOCK DATA FOR EXAMPLE: replace with actual df from your ingestion pipeline
    df = pd.DataFrame([{
        "registration_id": "REG123", "address": "123 Main St", "city": "Flagstaff",
        "bedrooms": 3, "atlas_segment": "Invest", "atlas_exploit_score": 85.5
    }])

    # 4. Push to Supabase
    print(f"Upserting {len(df)} records to Supabase...")
    data = df.to_dict(orient='records')
    try:
        # Use upsert based on the primary key (registration_id)
        response = supabase.table('str_properties').upsert(data).execute()
        print("Supabase upsert successful.")
    except Exception as e:
        print(f"Error pushing to Supabase: {e}")

def run_didre_scout_job():
    print("Starting DID Re Scout Job...")
    # [cite_start]This would call the logic from deal_scout_didre.py [cite: 613]
    # The scouts would use the Supabase client to check DNC and insert leads.
    # run_all_didre_agents(supabase_client=supabase)
    print("Scout job finished.")

if __name__ == "__main__":
    # In a real Fly.io setup, you might use a scheduler like Celery,
    # or just run this script via cron.
    run_str_ingestion_job()
    run_didre_scout_job()

#### Adapting the "AI Examination Bot" and "DID Re Scouts"

You need to adapt the provided Python code to use the `supabase` client instead of SQLAlchemy sessions.

[cite\_start]**Example: Adapting DID Re Lead Insertion[cite: 709]:**

In [ ]:
# In atlas_leads_store.py adaptation
def get_or_create_lead(supabase_client, *, agent_name, segment, source, source_id, title, url, score, **kwargs):
    # Prepare data payload
    data = {
        "agent_name": agent_name, "segment": segment, "source": source,
        "source_id": source_id, "title": title, "url": url, "score": score,
        **kwargs
    }
    # Supabase upsert handles "get or create" based on the unique constraint
    response = supabase_client.table('leads').upsert(data, on_conflict="source, source_id").execute()
    return response.data

-----

### Phase 3: The Frontend (Vite + React)

This replaces the Streamlit UI. It's "super fast" because it just fetches JSON from Supabase.

**Setup:**

```bash
npm create vite@latest atlas-frontend -- --template react-ts
cd atlas-frontend
npm install @supabase/supabase-js @tanstack/react-query react-router-dom tailwindcss postcss autoprefixer
# (Initialize TailwindCSS according to their docs)
```

#### 1\. Supabase Client (`src/lib/supabase.ts`)

```typescript
import { createClient } from '@supabase/supabase-js';

// Use the "Anon" public key here, not the service key.
const supabaseUrl = import.meta.env.VITE_SUPABASE_URL;
const supabaseAnonKey = import.meta.env.VITE_SUPABASE_ANON_KEY;

export const supabase = createClient(supabaseUrl, supabaseAnonKey);
```

#### 2\. Data Fetching Hook (`src/hooks/useSTRData.ts`)

We use React Query to fetch data and cache it. This makes subsequent page loads instant.

```typescript
import { useQuery } from '@tanstack/react-query';
import { supabase } from '../lib/supabase';

export const useSTRData = () => {
  return useQuery({
    queryKey: ['str_properties'],
    queryFn: async () => {
      // "Preload JSON": This is just a fast REST call to Supabase
      const { data, error } = await supabase
        .from('str_properties')
        .select('*')
        .order('atlas_exploit_score', { ascending: false })
        .limit(500); // Cap result size for speed

      if (error) throw error;
      return data;
    },
    staleTime: 1000 * 60 * 5, // Consider data fresh for 5 minutes
  });
};
```

#### 3\. The Dashboard Page (`src/pages/Dashboard.tsx`)

[cite\_start]This replaces the main Streamlit overview and dataframes[cite: 161, 170].

```tsx
import React from 'react';
import { useSTRData } from '../hooks/useSTRData';

const Dashboard = () => {
  // This hook handles loading states and caching automatically
  const { data: properties, isLoading, error } = useSTRData();

  if (isLoading) return <div className="p-8">Loading Atlas brain...</div>;
  if (error) return <div className="p-8 text-red-500">Error loading data.</div>;

  // Simple metric calculations on the client side
  const total = properties?.length || 0;
  const distressed = properties?.filter(p => p.atlas_segment === 'Distressed').length;

  return (
    <div className="p-6 bg-gray-100 min-h-screen">
      <h1 className="text-3xl font-bold mb-6">Atlas: Command Center</h1>

      [cite_start]{/* Metrics Section [cite: 165] */}
      <div className="grid grid-cols-1 md:grid-cols-4 gap-4 mb-8">
        <MetricCard title="Total STRs Tracks" value={total} />
        <MetricCard title="Distressed Targets" value={distressed} color="red" />
        {/* Add Invest/Divest metrics here */}
      </div>

      [cite_start]{/* Data Table Section [cite: 170] */}
      <div className="bg-white shadow rounded-lg overflow-hidden">
        <table className="min-w-full divide-y divide-gray-200">
          <thead className="bg-gray-50">
            <tr>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">Score</th>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">Segment</th>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">Address</th>
              <th className="px-6 py-3 text-left text-xs font-medium text-gray-500 uppercase tracking-wider">Beds</th>
            </tr>
          </thead>
          <tbody className="bg-white divide-y divide-gray-200">
            {properties?.slice(0, 50).map((prop) => (
              <tr key={prop.registration_id} className="hover:bg-gray-50">
                <td className="px-6 py-4 whitespace-nowrap font-bold">{prop.atlas_exploit_score?.toFixed(0)}</td>
                <td className="px-6 py-4 whitespace-nowrap">
                    <span className={`px-2 inline-flex text-xs leading-5 font-semibold rounded-full
                        ${prop.atlas_segment === 'Distressed' ? 'bg-red-100 text-red-800' :
                        prop.atlas_segment === 'Invest' ? 'bg-green-100 text-green-800' : 'bg-gray-100 text-gray-800'}`}>
                    {prop.atlas_segment}
                    </span>
                </td>
                <td className="px-6 py-4 whitespace-nowrap">{prop.address}</td>
                <td className="px-6 py-4 whitespace-nowrap">{prop.bedrooms}</td>
              </tr>
            ))}
          </tbody>
        </table>
      </div>
    </div>
  );
};

// Simple UI helper
const MetricCard = ({ title, value, color = "blue" }: any) => (
    <div className={`bg-white p-4 rounded-lg shadow border-l-4 border-${color}-500`}>
        <h3 className="text-gray-500 text-sm uppercase">{title}</h3>
        <p className="text-2xl font-bold">{value}</p>
    </div>
);

export default Dashboard;
```

### Deployment on Fly.io

You will have two separate Fly apps.

**1. The Frontend App (`fly.toml` for Vite app)**
This is a simple static site host.

```toml
app = "atlas-frontend"
primary_region = "lax"

[build]
  # Use a standard Node/Nginx builder for static sites
  builder = "paketobuildpacks/builder:base"
  buildpacks = ["gcr.io/paketo-buildpacks/nodejs", "gcr.io/paketo-buildpacks/nginx"]

[[services]]
  internal_port = 8080
  protocol = "tcp"
  [services.concurrency]
    hard_limit = 25
    soft_limit = 20
```

**2. The Backend Worker App (`fly.toml` for Python)**
This runs your Python script continuously or via cron. It doesn't need open ports if it's just a worker.

```toml
app = "atlas-worker"
primary_region = "lax"

[build]
  dockerfile = "Dockerfile.worker" # You'll need to write this

[env]
  # Set your Supabase Service Key and DB URL here as secrets
  # fly secrets set SUPABASE_SERVICE_KEY=...
```